In [1]:
import joblib
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import argparse
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
import math
import wandb

from wandb.integration.keras.callbacks import WandbMetricsLogger, WandbModelCheckpoint



data = np.load("../Models/pca-classifier-train-data.npy")
Y = np.load("../Data/LPS/sal-lps-150-1500(labeled)_labels.npy")
print(data.shape)

2025-09-13 19:30:59.611725: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-13 19:30:59.653772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-13 19:31:00.661266: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


(3274, 500)


In [2]:
wandb.init(
    project="PCAClassifier",
    config={
        "hidden_size": 64,
        "activation": "relu",
        "dropout": 0.5,
        "output_activation": "softmax",
        "optimizer": "adam",
        "loss": "sparse_categorical_crossentropy",
        "metric": "accuracy",
        "epochs": 50,
        "batch_size": 32,
        "learning_rate": 0.001
    }
)

config = wandb.config

wandb: Currently logged in as: trapezium (trapezium-wits-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
class MLPClassifier(tf.keras.Model):
    def __init__(self, input_dim, num_classes=2, hidden_size=64, dropout_rate=0.5):
        super().__init__()
        self.hidden = layers.Dense(hidden_size, activation='relu')
        self.dropout = layers.Dropout(dropout_rate)
        self.classifier = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.hidden(inputs)
        x = self.dropout(x)
        return self.classifier(x)

In [4]:
X_train, X_temp, y_train, y_temp = train_test_split(data, Y, test_size=0.3, random_state=42, stratify=Y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp)
print(f'Training data shape: {X_train.shape}')
print(f'Val data shape: {X_val.shape}')
print(f'Test data shape: {X_test.shape}')

Training data shape: (2291, 500)
Val data shape: (327, 500)
Test data shape: (656, 500)


In [ ]:
input_dim = X_train.shape[1]
model = MLPClassifier(input_dim=input_dim,hidden_size=config.hidden_size)

2025-09-13 19:31:04.963154: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


: 

In [ ]:
early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=3,
    restore_best_weights=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
            )

print("Training started....")
history = model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,    
    validation_data=(X_val, y_val), 
    verbose=1,
    callbacks=[early_stopping,
               WandbMetricsLogger(),
               WandbModelCheckpoint("nmf_model_{epoch:02d}_{batch:02d}.keras",save_best_only=True,monitor='accuracy')
               ]
)
wandb.finish()
print("Training completed!")

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay

loss, accuracy = model.evaluate(X_test, y_test)

y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
print(f'TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}')

if len(np.unique(y_test)) == 2:
    auc = roc_auc_score(y_test, y_pred_probs[:, 1])
    print(f'AUC: {auc}')
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs[:, 1])
    import matplotlib.pyplot as plt
    plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

# Confusion matrix plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
print(f'Test loss: {loss}')
print(f'Test Accuracy {accuracy}')